# AI Project Code - Viterbi Algorithm for POS Tagging

## Group Members :  
18ucc002 : Deepesh Kanodia  
18ucc058 : Naman Maheshwari  
18ucc160 : Manav Ladha  
18ucs179 : Shreena Parekh  




The entire code is majorly divided into 5 segments:

1. **Importing Necessaries**   
    1.1 Importing Libraries  
    1.2 Importing Dataset
  
2. **Data Preprocessing**   
    2.1 Extracting from .txt file   
    2.2 Removing empty list  
    2.3 Converting to Tuple  
    2.4 Unique Tags  
    2.5 Unique Words - Vocabulary  
  
3. **Computing Probabilities**  
    3.1 Map Tag to Index     
    3.2 Compute Word to Tag Count Matrix   
    3.3 Compute Tag to Tag Count Matrix    
    3.4 Compute Emission Probability Matrix   
    3.5 Compute Transition Probability Matrix   
  
4. **Viterbi Algorithm**   
    4.1 Viterbi Decoding Function  
    4.2 Backtracking Function  
    4.3 Map output state sequence   
  
5. **Testing**   
    5.1 Test code on an exapmle  
    5.2 Test code on entire testing set  


## 1.1 Importing Libraries

In [49]:
import nltk
import numpy as np
import pandas as pd
import random
import pprint, time
from collections import defaultdict
import statistics 
from scipy import stats

## 1.2 Importing Datasets

In [50]:
test_file = open("test.txt", "r")
train_file = open("train.txt", "r")

## 2.1 Extracting from .txt

In [51]:
train_list_of_lists = []
for line in train_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    train_list_of_lists.append(line_list)

train_file.close()

In [52]:
train_list_of_lists[30:50]
print("Number of Training Examples : ",len(train_list_of_lists))

Number of Training Examples :  220663


In [53]:
test_list_of_lists = []
for line in test_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    test_list_of_lists.append(line_list)

test_file.close()

In [54]:
test_list_of_lists[30:50]
print("Number of Testing Examples : ",len(test_list_of_lists))

Number of Testing Examples :  49389


## 2.2 Removing empty list

In [55]:

train_list_of_lists = [ele for ele in train_list_of_lists if ele!=[]]
print("Number of Training Examples after removing empty lists : ",len(train_list_of_lists))


Number of Training Examples after removing empty lists :  211727


## 2.3 Converting to Tuple

In [56]:
final_train_list_pos =[]
for i in train_list_of_lists :
    k = i[:2]
    j = tuple(k)
    final_train_list_pos.append(j)

In [57]:
final_test_list_pos =[]
for i in test_list_of_lists :
    k = i[:2]
    j = tuple(k)
    final_test_list_pos.append(j)

In [58]:
print(len(final_train_list_pos))
final_train_list_pos[:10]

211727


[('Confidence', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('pound', 'NN'),
 ('is', 'VBZ'),
 ('widely', 'RB'),
 ('expected', 'VBN'),
 ('to', 'TO'),
 ('take', 'VB'),
 ('another', 'DT')]

In [59]:
print(len(final_test_list_pos))
final_test_list_pos[:10]

49389


[('Rockwell', 'NNP'),
 ('International', 'NNP'),
 ('Corp.', 'NNP'),
 ("'s", 'POS'),
 ('Tulsa', 'NNP'),
 ('unit', 'NN'),
 ('said', 'VBD'),
 ('it', 'PRP'),
 ('signed', 'VBD'),
 ('a', 'DT')]

## 2.4 Unique POS tags 

In [60]:
unique_pos_tags = list({tag for word,tag in final_train_list_pos})
print(unique_pos_tags)

['NN', 'RB', ',', 'VBP', 'JJS', '(', 'PRP$', 'VBN', 'WP', 'FW', '.', 'EX', '$', 'PRP', 'WP$', 'WDT', 'VBZ', 'VB', 'VBG', 'JJ', 'CD', '#', ')', 'PDT', 'MD', 'RP', 'CC', 'SYM', 'IN', 'RBS', "''", 'NNP', 'UH', 'VBD', 'NNS', ':', 'TO', 'JJR', 'WRB', 'RBR', 'POS', 'DT', '``', 'NNPS']


In [13]:
print("Number of Unique POS Tag in Training Set : ",len(unique_pos_tags))

Number of Unique POS Tag in Training Set :  44


## 2.5 Unique Words - Vocabulary

In [14]:
# check total words in vocabulary
vocab = {word for word,tag in final_train_list_pos}
print("Number of Unique Words in Training Set : ",len(vocab))


Number of Unique Words in Training Set :  19122


## 3.1 Map index to Tag

In [15]:
# Function to map the state tags to unique indices i.e. 0 : NN , 1 : DT

def assign_index_tag(unique_pos_tags):
    tag_map={}
    vals = list(range(0,len(unique_pos_tags)))
    
    for i , tag in enumerate(unique_pos_tags):
        #tag_map[tag] = i
        tag_map[i] = tag
        
    #tag_map[unique_pos_tags]=vals
    return tag_map

In [16]:
tag_map = assign_index_tag(unique_pos_tags)
print(tag_map)

{0: 'NN', 1: 'RB', 2: ',', 3: 'VBP', 4: 'JJS', 5: '(', 6: 'PRP$', 7: 'VBN', 8: 'WP', 9: 'FW', 10: '.', 11: 'EX', 12: '$', 13: 'PRP', 14: 'WP$', 15: 'WDT', 16: 'VBZ', 17: 'VB', 18: 'VBG', 19: 'JJ', 20: 'CD', 21: '#', 22: ')', 23: 'PDT', 24: 'MD', 25: 'RP', 26: 'CC', 27: 'SYM', 28: 'IN', 29: 'RBS', 30: "''", 31: 'NNP', 32: 'UH', 33: 'VBD', 34: 'NNS', 35: ':', 36: 'TO', 37: 'JJR', 38: 'WRB', 39: 'RBR', 40: 'POS', 41: 'DT', 42: '``', 43: 'NNPS'}


## 3.2 Compute Word to Tag Count Matrix

In [17]:
# Function to compute the word to tag count matrix which stores the count of each word w being in each state t

def compute_count_matrix_wordtotag(vocab,unique_pos_tags,train_set):
    
    count_mat_wt = pd.DataFrame(0,columns = list(vocab), index=list(unique_pos_tags))
    count_mat_t = pd.DataFrame(0,columns = ['count'], index=list(unique_pos_tags))
    
    for w,t in train_set :
        count_mat_t.loc[t,'count']+=1
        count_mat_wt.loc[t,w]+=1  
  
    return count_mat_t,count_mat_wt
        
        
    

In [18]:
count_mat_t,count_mat_wt = compute_count_matrix_wordtotag(vocab,unique_pos_tags,final_train_list_pos)

In [ ]:
print("WORD to TAG COUNT MATRIX")
count_mat_wt

In [23]:
print("TAG COUNT MATRIX")
print(count_mat_t)

TAG COUNT MATRIX
      count
NN    30147
RB     6607
,     10770
VBP    2868
JJS     374
(       274
PRP$   1881
VBN    4763
WP      529
FW       38
.      8827
EX      206
$      1750
PRP    3820
WP$      35
WDT     955
VBZ    4648
VB     6017
VBG    3272
JJ    13085
CD     8315
#        36
)       281
PDT      55
MD     2167
RP       83
CC     5372
SYM       6
IN    22764
RBS     191
''     1493
NNP   19884
UH       15
VBD    6745
NNS   13619
:      1047
TO     5081
JJR     853
WRB     478
RBR     321
POS    1769
DT    18335
``     1531
NNPS    420


## 3.3 Compute Tag to Tag Count Matrix

In [24]:
# Function to compute the tag to tag count matrix which stores the count of each tag t1 being preceded by each state t2

def compute_count_matrix_tagtotag(vocab,unique_pos_tags,train_set):
    
    count_mat_tt = pd.DataFrame(0,columns = list(unique_pos_tags), index=list(unique_pos_tags))
    
    for i in range(0,len(train_set)-1) :
        cur_tag = train_set[i][1]
        nxt_tag = train_set[i+1][1]
        count_mat_tt.loc[cur_tag,nxt_tag]+=1   
  
    return count_mat_tt
                

In [25]:
count_mat_tt = compute_count_matrix_tagtotag(vocab,unique_pos_tags,final_train_list_pos)

In [ ]:
print("TAG to TAG COUNT MATRIX")
count_mat_tt

## 3.4 Compute Emission Probability Matrix

In [27]:
# Function to compute the emission probability matrix using the word to tag count matrix and tag count matrix

def compute_emission_matrix(count_mat_t,count_mat_wt):
    emission_mat = pd.DataFrame(0,columns = list(vocab), index=list(unique_pos_tags))
    
    for t in unique_pos_tags:
        
        for w in vocab:
            emission_mat.loc[t,w]=count_mat_wt.loc[t,w]/count_mat_t.loc[t,'count']
    
    return emission_mat
    


In [28]:
emmision_mat = compute_emission_matrix(count_mat_t,count_mat_wt)

In [ ]:
print("EMISSION MATRIX")
emmision_mat[1:10][1:10]

In [ ]:
emmision_mat.loc['NNP','September']

## 3.5 Compute Transition Probability Matrix

In [30]:
# Function to compute the transition probability matrix using the tag to tag count matrix and tag count matrix

def compute_transition_matrixx(count_mat_t,count_mat_tt):
    
    transition_mat = pd.DataFrame(0,columns = list(unique_pos_tags), index=list(unique_pos_tags))
    
    for t1 in unique_pos_tags:
        for t2 in unique_pos_tags:
             transition_mat.loc[t1,t2]=count_mat_tt.loc[t1,t2]/count_mat_t.loc[t1,'count']
    
    return  transition_mat
    

In [31]:
transition_mat = compute_transition_matrixx(count_mat_t,count_mat_tt)

In [ ]:
transition_mat

In [44]:
print(transition_mat['POS']['NNS'])

0.009839195241941405


## 4.1 Viterbi Algorithm

In [83]:
#Function to compute the trelis matrix and backpointer matrix to keep track of maximum probablity value and path

def Viterbi (tag_map , emmision_mat , transition_mat , init_dist , end_dist , observation):    
        
    N = transition_mat.shape[0]
    T = len(observation)
    
    rows = N + 2
    cols = T + 1
     
    trelis = np.zeros((rows, cols),dtype='float64')    
    backpointer = np.zeros((rows, cols),dtype='float64')
    start_symbol = observation[0]
    
    transition_matrix = transition_mat.values
    if start_symbol not in emmision_mat.columns:
            emmision_mat[start_symbol] = 0.00001
        
    #INITIALIZATOIN
   
    for s in range(0,N):
        
        trelis[s][0] = init_dist[s] * emmision_mat.loc[tag_map[s] ,start_symbol] 
        backpointer[s][0] = 0
        
        
    #TRAVERSING
    
    for t in range(1,T):
        
        cur_obs = observation[t]
        if cur_obs not in emmision_mat.columns:
            emmision_mat[cur_obs]=0.00001
           
        for s in range(0,N):
           
            max_val = -1
            max_pos = -1
            for pre_s in range(0,N):
                pre_viterbi = trelis[pre_s][t-1]
                transition_val = transition_matrix[pre_s][s]
                
                val = pre_viterbi * transition_val
                
                if val>max_val:
                    max_val = val
                    max_pos = pre_s
                    
            trelis[s][t] = max_val * emmision_mat.loc[tag_map[s],cur_obs]
            backpointer[s][t] = max_pos
            
   
     
    #TERMINATING
    
    max_end_value = -1
    max_end_position = -1
    for s in range(0,N):
        
        end_viterbi = trelis[s][T-1]
        end_transition_val = end_dist[s]
        
        end_value = end_viterbi * end_transition_val
        
        if end_value > max_end_value:
            max_end_value = end_value
            max_end_pos = s
            
    trelis[N][T] = max_end_value
    backpointer[N][T] = max_end_pos
    
    return trelis , backpointer

## 4.2 Backtrace Function

In [84]:
#Function to find the optimal sequence of state indices from the Backpointer matrix

def backtrace_seq(backpointer):

    last_state = backpointer[-2][-1]
    back_df = pd.DataFrame(data = backpointer)  

    max_occur_list = []
    
    for col in back_df.columns[1:-1]:
        
        max_occur = int(stats.mode(list(back_df[col]))[0])
        max_occur_list.append(max_occur)
   
    
    max_occur_list.append(int(last_state))
    return max_occur_list

## 4.3 Map Output Sequence

In [85]:
#Function to map the sequence of state indices to actual state tag names

def index_output_seq(output_seq , tag_map):
    optimal_states = []
    for index in output_seq:
        optimal_states.append(tag_map[index])
        
    return optimal_states

## 5.1 Testing Algorithm on an Example

In [86]:
#Setting initial parameteres like number of states , initial probability distribution and final probability distribution

N = transition_mat.shape[0]
init_dist = list(transition_mat.loc['.',:])
end_dist = list(transition_mat['.'])

observation = ['I','have','a','dog']
original_result = ['PRP','VBP','DT','NN']

In [87]:
start = time.time()
trelis , backpointer = Viterbi (tag_map , emmision_mat , transition_mat , init_dist , end_dist , observation)
max_occur_list = backtrace_seq(backpointer)
output_state_seq = index_output_seq(max_occur_list , tag_map)
end = time.time()

In [88]:
max_occur_list

[13, 3, 41, 0]

In [89]:
output_state_seq

['PRP', 'VBP', 'DT', 'NN']

In [90]:
correctly_labeled = 0
for i in range(len(observation)):
    if output_state_seq[i]==original_result[i]:
        correctly_labeled+=1
print("ACCURACY OF THE ALGORITHM : ",correctly_labeled/len(observation)*100,"%")
print("TIME TAKEN : ",end-start)

ACCURACY OF THE ALGORITHM :  100.0 %
TIME TAKEN :  0.012966632843017578


## 5.2 Testing Algorithm on entire dataset

In [ ]:
#Function to preprocess the test dataset into list of sentences to be fed to the Viterbi Algorithm

def generate_test_data(test_set):

    test_sentences=[]
    test_original_outputs=[]
    sentence = []
    output = []
    for test_case in test_set:
        if test_case:
            sentence.append(test_case[0])
            output.append(test_case[1])
        else:
            
            test_sentences.append(sentence)
            test_original_outputs.append(output)
            sentence=[]
            output=[]
    return test_sentences , test_original_outputs



In [ ]:
test_sentences , test_original_outputs = generate_test_data(final_test_list_pos)
print("Number of test sentences : ",len(test_sentences))

In [ ]:
"""
For each of the test sentence we print :
    1. x - The input observation
    2. y - The original output state sequence
    3. y_pred - The predicted output state sequence
"""
 
i=1
for x,y in zip(test_sentences , test_original_outputs):
    print("------------TEST CASE ",i," ------------------")
    print("Input x : ",x)
    print("Original Output y : ",y)
    
    trelis , backpointer = Viterbi (tag_map , emmision_mat , transition_mat , init_dist , end_dist , x)
    max_occur_list = backtrace_seq(backpointer)
    y_pred = index_output_seq(max_occur_list , tag_map)

    print("Predicted Output y_pred : ",y_pred)
    print()
    i+=1